In [86]:
### ETAPA 1: COSECHA DE UVA ###
# Input: 290 lotes
# Output: lotes cosechados para proceso fermentación.

# Activación base de datos y librerías
import sqlite3
import random
import numpy as np
import openpyxl

wb = openpyxl.Workbook()
ws = wb.active

conn = sqlite3.connect('vinos.db')
c = conn.cursor()

In [87]:
# parametros cosecha
periodo_ideal = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
calidades =  {'C1': [0.85, 0.95], 'C2': [0.92, 0.93], 'C3': [0.91, 0.87], 'C4': [0.95, 0.95], 'C5': [0.85, 0.85], 'C6': [0.93, 0.94]}
umbrales = {'C1': 0.8, 'C2': 0.75, 'C3': 0.8, 'C4': 0.7, 'C5': 0.8, 'C6': 0.7}
lotes = []
penalizaciones = {
    'C1': [0.03, 0.02, 0.01, 0.005],
    'C2': [0.025, 0.012, 0.004, 0.002],
    'C3': [0.03, 0.02, 0.01, 0.005],
    'C4': [0.025, 0.012, 0.01, 0.002],
    'C5': [0.03, 0.02, 0.01, 0.005],
    'C6': [0.025, 0.012, 0.004, 0.002]
}

In [88]:
# Leo la base de datos y la guardo en una lista
c.execute("SELECT * FROM lotes")
lotes_obtenidos = c.fetchall()
lotes.append(lotes_obtenidos)
lotes = lotes[0]

In [89]:
#Actualizo el día óptimo de cosecha según la distribución normal de cada lote en la base de datos

# pasar lotes a lista de listas
lotes = [list(elem) for elem in lotes]
# Funcion que estima el dia de cosecha con media 0 y desv est 2
def dia_estimado():
        optimal_day = np.random.normal(0, 2)
        return int(round(optimal_day))
for lote in lotes:
    variacion_optimo = dia_estimado()
    lote[5]+=variacion_optimo
    lote.append(variacion_optimo)


In [90]:
# Lotes actualizados
print(lotes)

[[1, 'L_1_C4', 1, 'C4', 217.5, 115, 0.43, 0.39, 0.257, -3], [2, 'L_2_C4', 2, 'C4', 517.5, 121, 0.35, 0.44, 0.269, -2], [3, 'L_3_C6', 3, 'C6', 450.0, 77, 0.46, 0.38, 0.362, -2], [4, 'L_4_C4', 4, 'C4', 168.75, 118, 0.32, 0.51, 0.276, -4], [5, 'L_5_C6', 5, 'C6', 243.75, 102, 0.49, 0.6, 0.348, -2], [6, 'L_6_C3', 6, 'C3', 150.0, 96, 0.31, 0.56, 0.579, 0], [7, 'L_7_C5', 7, 'C5', 558.75, 108, 0.43, 0.39, 0.795, -1], [8, 'L_8_C6', 8, 'C6', 442.5, 78, 0.38, 0.35, 0.36, 0], [9, 'L_9_C1', 9, 'C1', 300.0, 123, 0.36, 0.35, 0.528, -2], [10, 'L_10_C5', 10, 'C5', 108.75, 81, 0.5, 0.56, 0.806, 2], [11, 'L_11_C1', 11, 'C1', 555.0, 100, 0.31, 0.53, 0.557, 0], [12, 'L_12_C1', 12, 'C1', 412.5, 125, 0.41, 0.33, 0.516, 1], [13, 'L_13_C5', 13, 'C5', 547.5, 87, 0.5, 0.48, 0.811, -2], [14, 'L_14_C2', 14, 'C2', 498.75, 117, 0.44, 0.52, 0.416, 1], [15, 'L_15_C1', 15, 'C1', 101.25, 125, 0.31, 0.48, 0.554, -2], [16, 'L_16_C6', 16, 'C6', 502.5, 96, 0.47, 0.34, 0.361, -1], [17, 'L_17_C5', 17, 'C5', 536.25, 112, 0.45,

In [91]:
# Función para predecir el clima con markov
def predecir_clima(prob_seco_lluvia, prob_lluvia_lluvia):
    clima_prediccion = []  # Lista para almacenar la predicción
    tipo_3 = []
    # Inicializar el primer día como seco
    clima_prediccion.append(0)
    tipo_3.append(0)
    # Calcular predicciones para los siguientes 18 días (13 días previos + 5 días de desviación)
    for _ in range(7):
        if clima_prediccion[-1] == 0:  # Si el día anterior fue seco
            clima_prediccion.append(1 if random.random() < prob_seco_lluvia else 0)
            tipo_3.append(0)
        else:  # Si el día anterior fue lluvioso
            clima_prediccion.append(1 if random.random() < prob_lluvia_lluvia else 0)
            if clima_prediccion[-1] == 1:
                tipo_3.append(1 if random.random() <= 0.05 else 0)
    if sum(tipo_3) >= 1:
        perdida = True
    else:
        perdida = False
    return clima_prediccion, perdida

In [92]:
def obtener_penalizaciones(tipo_uva, calidad, clima_prediccion):
    lista_penalizaciones = penalizaciones[tipo_uva]
    index = 7
    while index != 0:
        if index == 7:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[0])
        elif index == 6:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[1])
        elif index == 5:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[2])
        elif index == 4:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        elif index == 3:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        elif index == 2:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        elif index == 1:
            if clima_prediccion[index] == 1:
                calidad = calidad * (1 - lista_penalizaciones[3])
        index -= 1
    return calidad


In [93]:
# funcion que entrega el valor final de la calidad considerando la lluvia
def calcular_calidad(q_t_minus, q_t_plus, t_optimal, clima_prediccion, tipo_uva):
    t_minus = -7
    t_plus = 7
    A = np.array([
        [t_minus**2, t_minus, 1],
        [0, 0, 1],
        [t_plus**2, t_plus, 1]])
    b = np.array([q_t_minus, 1, q_t_plus])
    coefficients = np.linalg.solve(A, b)
    a, b, c = coefficients
    def funcion_calidad(a, b, c, t, clima_prediccion, tipo_uva):
        # funcion de calidad bajo el supuesto que no llueve, para modificar en el futuro
        calidad_sin_lluvia = max(min(a * t**2 + b * t + c, 1), 0)
        calidad_final = obtener_penalizaciones(tipo_uva, calidad_sin_lluvia, clima_prediccion)
        return calidad_final
    q_t_optimal = funcion_calidad(a, b, c, t_optimal, clima_prediccion, tipo_uva)
    return q_t_optimal

In [94]:
# Parametros para encontrar en la simulacion etapa 1
calidades_lotes = []
lotes_perdidos = []
lotes_obtenidos = []
litros_totales_perdidos = 0
litros_totales_obtenidos = 0
botellas_totales_perdidas = 0
botellas_totales_obtenidas = 0
costos_totales_perdidos = 0
costos_totales_obtenidos = 0

In [95]:
# Simulación de la etapa 1
for t in range(50, 150):
    for lote in lotes:
        lote_id = lote[0]
        lote_cod = lote[1]
        lote_numero = lote[2]
        lote_tipo_uva = lote[3]
        lote_toneladas = lote[4]
        lote_dia_optimo = lote[5]
        lote_prob_seco_lluvia = lote[6]
        lote_prob_lluvia_lluvia = lote[7]
        lote_precio_usd = lote[8]
        lote_variacion = lote[9]
        if t == lote_dia_optimo - 8:
            lote_clima_prediccion, perdida = predecir_clima(lote_prob_seco_lluvia, lote_prob_lluvia_lluvia)
            if perdida == True:
                lotes_perdidos.append(lote)
                pass
            elif not perdida:
                lista = lote
                if sum(lote_clima_prediccion) > 4:
                    # compra forward
                    lista.append(0)
                elif sum(lote_clima_prediccion) <= 4:
                    # compra spot
                    lista.append(1)
                q_t = calidades[lote_tipo_uva]
                lote_q_t_minus = q_t[0]
                lote_q_t_plus = q_t[1]
                calidad_lote = calcular_calidad(lote_q_t_minus, lote_q_t_plus, lote_variacion, lote_clima_prediccion, lote_tipo_uva)
                calidades_lotes.append(calidad_lote)
                lista.append(calidad_lote)
                lotes_obtenidos.append(lista)
                
                

In [96]:
# Modificamos los lotes obtenidos y perdidos con la info relevante, es decir, 
# sacamos las probabilidades de lluvia y el día óptimo de cosecha, entre otras cosas.
lista_final = []
for lote in lotes_obtenidos:
    lista = []
    lote_cod = lote[1]
    lote_numero = lote[2]
    lote_tipo_uva = lote[3]
    lote_toneladas = lote[4]
    lote_costo = lote[8]
    lote_calidad = lote[11]
    lote_contrato = lote[10]
    lista.append(lote_cod)
    lista.append(lote_numero)
    lista.append(lote_tipo_uva)
    lista.append(lote_toneladas)
    lista.append(lote_costo)
    lista.append(lote_calidad)
    lista.append(lote_contrato)
    lista_final.append(lista)
print(lotes_obtenidos)
print(lista_final)
print(lotes_perdidos)
lista_final_perdidos = []
for lote in lotes_perdidos:
    lista = []
    lote_cod = lote[1]
    lote_numero = lote[2]
    lote_tipo_uva = lote[3]
    lote_toneladas = lote[4]
    lote_costo = lote[8]
    lote_calidad = 0
    lote_contrato = 1
    lista.append(lote_cod)
    lista.append(lote_numero)
    lista.append(lote_tipo_uva)
    lista.append(lote_toneladas)
    lista.append(lote_costo)
    lista.append(lote_calidad)
    lista.append(lote_contrato)
    lista_final_perdidos.append(lista)

[[20, 'L_20_C4', 20, 'C4', 225.0, 70, 0.33, 0.41, 0.274, -4, 1, 0.9475918346938775], [126, 'L_126_C5', 126, 'C5', 187.5, 70, 0.36, 0.35, 0.831, -1, 1, 0.9476899999999999], [22, 'L_22_C1', 22, 'C1', 420.0, 71, 0.39, 0.35, 0.554, -1, 1, 0.9467235137755102], [125, 'L_125_C4', 125, 'C4', 138.75, 71, 0.33, 0.6, 0.272, -2, 1, 0.9919386775510204], [194, 'L_194_C4', 194, 'C4', 502.5, 71, 0.35, 0.32, 0.27, -2, 1, 0.980035413420408], [120, 'L_120_C3', 120, 'C3', 217.5, 72, 0.42, 0.46, 0.605, 1, 1, 0.9458703901147959], [187, 'L_187_C2', 187, 'C2', 318.75, 72, 0.33, 0.58, 0.401, -1, 1, 0.9937680726530612], [275, 'L_275_C4', 275, 'C4', 382.5, 72, 0.31, 0.54, 0.276, -1, 0, 0.9712709045031879], [175, 'L_175_C1', 175, 'C1', 517.5, 73, 0.41, 0.57, 0.55, 2, 1, 0.9605222549999999], [178, 'L_178_C6', 178, 'C6', 363.75, 73, 0.37, 0.48, 0.346, 0, 1, 0.9840519520000001], [238, 'L_238_C5', 238, 'C5', 198.75, 73, 0.32, 0.42, 0.808, 2, 1, 0.9533318367346938], [259, 'L_259_C2', 259, 'C2', 127.5, 73, 0.42, 0.32, 

In [97]:
lotes_obtenidos = lista_final
lotes_perdidos = lista_final_perdidos
# Encontrar la info que queramos de esta etapa
print(f'Calidad máxima: {max(calidades_lotes)}')
print(f'Calidad mínima: {min(calidades_lotes)}')
print('')
for lote in lotes_perdidos:
    toneladas = lote[3]
    kilos = toneladas * 1000
    litros = kilos * 0.5
    litros_totales_perdidos += litros
    costo_lote = kilos*lote[4]
    if lote[6] == 0:
        costo_lote = costo_lote * 0.8
        costos_totales_perdidos += costo_lote
    elif lote[6] == 1:
        costo_lote = costo_lote
        costos_totales_perdidos += costo_lote

botellas_totales_perdidas = litros_totales_perdidos * 0.75
print(f'Litros totales perdidos: {round(litros_totales_perdidos)}')
print(f'Botellas totales perdidas: {round(botellas_totales_perdidas)}')
print(f'Costos totales perdidos USD (compra spot): {round(costos_totales_perdidos)}')
print('')
for lote in lotes_obtenidos:
    toneladas = lote[3]
    kilos = toneladas * 1000
    litros = kilos * 0.5
    litros_totales_obtenidos += litros
    costo_lote = kilos*lote[4]
    if lote[6] == 0:
        costo_lote = costo_lote * 0.8
        costos_totales_obtenidos += costo_lote
    elif lote[6] == 1:
        costos_totales_obtenidos += costo_lote*lote[5]
botellas_totales_obtenidas = litros_totales_obtenidos * 0.75
print(f'Litros totales obtenidos: {round(litros_totales_obtenidos)}')
print(f'Botellas totales obtenidas: {round(botellas_totales_obtenidas)}')
print(f'Costos totales esperadas USD (considerando contratos spot y foward): {round(costos_totales_obtenidos)}')
print('')
Costos_totales = costos_totales_obtenidos + costos_totales_perdidos
print(f'Costos totales etapa 1: {round(Costos_totales)}')


Calidad máxima: 1.0
Calidad mínima: 0.8248835749472999

Litros totales perdidos: 2341875
Botellas totales perdidas: 1756406
Costos totales perdidos USD (compra spot): 2320301

Litros totales obtenidos: 45838125
Botellas totales obtenidas: 34378594
Costos totales esperadas USD (considerando contratos spot y foward): 45417053

Costos totales etapa 1: 47737354


In [98]:
# Guardar los lotes obtenidos en la base de datos
# Ejecutar solo una vez
#import sqlite3



# Conectar a la base de datos (o crearla si no existe)
#conn = sqlite3.connect('etapa_1.db')

### AQUI SACAR # PARA QUE FUNCIONE ESTA CELDA Y SE PUEDA CORRER
#cursor = conn.cursor()

# Crear una tabla
#cursor.execute('''
#CREATE TABLE IF NOT EXISTS lotes (
#    lote_cod TEXT,
#    lote_numero INTEGER,
#    lote_tipo_uva TEXT,
#    lote_toneladas REAL,
#    lote_precio_usd REAL,
#    lote_calidad REAL,
 #   lote_contrato INTEGER
#)
#''')

# Insertar datos en la tabla
#for lote in lista_final:
#    cursor.execute('''
#    INSERT INTO lotes (
#        lote_cod, lote_numero, lote_tipo_uva, lote_toneladas, 
#        lote_precio_usd, lote_calidad, lote_contrato
#    ) VALUES (?, ?, ?, ?, ?, ?, ?)
#    ''', lote)

# Guardar los cambios y cerrar la conexión
#conn.commit()
#conn.close()


In [99]:
### ETAPA 2: CALIDAD DE UVA ###

# Parámetros simulación etapa 2
costo_recuperado = 0
lista_final = lotes_obtenidos + lotes_perdidos

In [100]:
contador = 0
for lote in lista_final:
    calidad = lote[5]
    cepa = lote[2]
    precio = lote[4]
    tipo_compra = lote[6]
    kilos = lote[3]*1000
    umbral = umbrales[cepa]
    #Obtenemos costo por lote dependiendo de como ha sido comprado
    if tipo_compra == 0:
        costo = precio * 0.8 * kilos
    else :
        costo = precio * kilos
    if 0.5 <= calidad < umbral:
        recuperado = costo * 0.3 
    elif calidad < 0.5:
        recuperado = costo * 0.05
        # recuperado = costo * calidad * 0.05
    else:
        contador += 1
        recuperado = 0
    costo_recuperado += recuperado
print(f'Costos recuperados etapa 2: {round(costo_recuperado)} USD')

Costos recuperados etapa 2: 116015 USD


### ETAPA 3: FERMENTACIÓN ###

In [ ]:
# calcula la duracion en fermentación
def calcular_duracion_vinificacion():
    duracion_aleatoria = random.uniform(7, 9)
    dia_exacto = round(duracion_aleatoria)
    return dia_exacto

In [ ]:
# Parámetros simulación etapa 3